MLflow is a versatile, expandable, open-source platform for managing workflows and artifacts across the machine learning lifecycle.

In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Step 1: Create imbalanced binary classification dataset
X, y = make_classification(
    n_samples=1000, n_features=10, n_informative=2, 
    n_redundant=8, weights=[.9, .1], flip_y=0, random_state=42
)

print(np.unique(y, return_counts=True))

# split the dataset in train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, stratify=y, random_state=42)

(array([0, 1]), array([900, 100]))


In [3]:
# Define model hyperparameters
params = {
    'solver': 'lbfgs',
    'max_iter': 1000,
    'multi_class': 'auto',
    'random_state': 42,
}

# train model
lr = LogisticRegression(**params, )
lr.fit(X_train, y_train)

# predict on test dataset
y_pred = lr.predict(X_test)

report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.95      0.97      0.96       270
           1       0.62      0.50      0.56        30

    accuracy                           0.92       300
   macro avg       0.79      0.73      0.76       300
weighted avg       0.91      0.92      0.92       300



In [4]:
report_dict = classification_report(y_test, y_pred, output_dict=True)
report_dict

{'0': {'precision': 0.9456521739130435,
  'recall': 0.9666666666666667,
  'f1-score': 0.9560439560439561,
  'support': 270.0},
 '1': {'precision': 0.625,
  'recall': 0.5,
  'f1-score': 0.5555555555555556,
  'support': 30.0},
 'accuracy': 0.92,
 'macro avg': {'precision': 0.7853260869565217,
  'recall': 0.7333333333333334,
  'f1-score': 0.7557997557997558,
  'support': 300.0},
 'weighted avg': {'precision': 0.9135869565217392,
  'recall': 0.92,
  'f1-score': 0.9159951159951161,
  'support': 300.0}}

In [5]:
import mlflow

mlflow.set_experiment('test exp')
mlflow.set_tracking_uri('http://127.0.0.1:5000/')

with mlflow.start_run(run_name='experiment logreg 1'):
    mlflow.log_params(params)
    mlflow.log_metrics({
        'accuracy': report_dict['accuracy'],
        'recall_class_0': report_dict['0']['recall'],
        'recall_class_1': report_dict['1']['recall'],
        'f1_score_macro': report_dict['macro avg']['f1-score']
    })
    mlflow.sklearn.log_model(lr, 'Logistic Regression')

2024/09/21 12:28:37 INFO mlflow.tracking.fluent: Experiment with name 'test exp' does not exist. Creating a new experiment.
2024/09/21 12:28:41 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/09/21 12:28:41 INFO mlflow.tracking._tracking_service.client: 🏃 View run experiment logreg 1 at: http://127.0.0.1:5000/#/experiments/134716432819852541/runs/7626f93b87254dbaa166c06d71ae6edb.
2024/09/21 12:28:41 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/134716432819852541.


To access the experiment open the terminal in the same folder the `mlruns` folder is and run `mlflow ui`

In [6]:
# Experiment 1 - Linear Regression
log_reg = LogisticRegression(C=1, solver='liblinear')
log_reg.fit(X_train, y_train)
y_pred_log_reg = log_reg.predict(X_test)
print(classification_report(y_test, y_pred_log_reg))

              precision    recall  f1-score   support

           0       0.95      0.96      0.95       270
           1       0.60      0.50      0.55        30

    accuracy                           0.92       300
   macro avg       0.77      0.73      0.75       300
weighted avg       0.91      0.92      0.91       300



In [7]:
# Experiment 2 - Random Forest Classifier
rf_clf = RandomForestClassifier(n_estimators=30, max_depth=3)
rf_clf.fit(X_train, y_train)
y_pred_rf_clf = rf_clf.predict(X_test)
print(classification_report(y_test, y_pred_rf_clf))

              precision    recall  f1-score   support

           0       0.97      1.00      0.98       270
           1       0.95      0.70      0.81        30

    accuracy                           0.97       300
   macro avg       0.96      0.85      0.89       300
weighted avg       0.97      0.97      0.96       300



In [8]:
# Experiment 3 - XGBoost
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train, y_train)
y_pred_xgb_clf = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb_clf))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       270
           1       0.96      0.80      0.87        30

    accuracy                           0.98       300
   macro avg       0.97      0.90      0.93       300
weighted avg       0.98      0.98      0.98       300



In [9]:
# Experiment 4 - Handle class imbalance using SMOTETomek and then train XGBoost
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)

print(np.unique(y_train_res, return_counts=True))

xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train_res, y_train_res)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

(array([0, 1]), array([619, 619]))
              precision    recall  f1-score   support

           0       0.98      0.98      0.98       270
           1       0.81      0.83      0.82        30

    accuracy                           0.96       300
   macro avg       0.89      0.91      0.90       300
weighted avg       0.96      0.96      0.96       300



In [10]:
# Track experiments using MLFlow
models = [
    (
        'Logistic Regression',
        {'C': 1, 'solver': 'liblinear'},
        LogisticRegression(),
        (X_train, y_train),
        (X_test, y_test),
    ),
    (
        'Random Forest',
        {'n_estimators': 30, 'max_depth': 3},
        RandomForestClassifier(),
        (X_train, y_train),
        (X_test, y_test),
    ),
    (
        'XGBoost Classifier',
        {'use_label_encoder': False, 'eval_metric': 'logloss'},
        XGBClassifier(),
        (X_train, y_train),
        (X_test, y_test),
    ),
    (
        'XGBoost Classifier with SMOTE',
        {'use_label_encoder': False, 'eval_metric': 'logloss'},
        XGBClassifier(),
        (X_train_res, y_train_res),
        (X_test, y_test),
    )
]

reports = []
for model_name, params, model, train_set, test_set in models:
    X_test, y_test = test_set

    model.set_params(**params)
    model.fit(*train_set)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

reports

[{'0': {'precision': 0.9454545454545454,
   'recall': 0.9629629629629629,
   'f1-score': 0.9541284403669725,
   'support': 270.0},
  '1': {'precision': 0.6,
   'recall': 0.5,
   'f1-score': 0.5454545454545454,
   'support': 30.0},
  'accuracy': 0.9166666666666666,
  'macro avg': {'precision': 0.7727272727272727,
   'recall': 0.7314814814814814,
   'f1-score': 0.749791492910759,
   'support': 300.0},
  'weighted avg': {'precision': 0.9109090909090909,
   'recall': 0.9166666666666666,
   'f1-score': 0.91326105087573,
   'support': 300.0}},
 {'0': {'precision': 0.9676258992805755,
   'recall': 0.9962962962962963,
   'f1-score': 0.9817518248175182,
   'support': 270.0},
  '1': {'precision': 0.9545454545454546,
   'recall': 0.7,
   'f1-score': 0.8076923076923077,
   'support': 30.0},
  'accuracy': 0.9666666666666667,
  'macro avg': {'precision': 0.961085676913015,
   'recall': 0.8481481481481481,
   'f1-score': 0.8947220662549129,
   'support': 300.0},
  'weighted avg': {'precision': 0.9663

In [11]:
mlflow.set_experiment("Anomaly Detection")
mlflow.set_tracking_uri('http://127.0.0.1:5000/')

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]

    with mlflow.start_run(run_name = model_name):
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_0': report['0']['recall'],
            'recall_class_1': report['1']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })
        
        if 'XGB' in model_name:
            mlflow.xgboost.log_model(model, 'model')
        else:
            mlflow.sklearn.log_model(model, 'model')

2024/09/21 12:28:43 INFO mlflow.tracking.fluent: Experiment with name 'Anomaly Detection' does not exist. Creating a new experiment.
2024/09/21 12:28:46 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/09/21 12:28:46 INFO mlflow.tracking._tracking_service.client: 🏃 View run Logistic Regression at: http://127.0.0.1:5000/#/experiments/686724081783997741/runs/7bc2c832acbd4a118ecde9ebe90d95c5.
2024/09/21 12:28:46 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/686724081783997741.
2024/09/21 12:28:49 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/09/21 12:28:49 INFO mlflow.tracking._tracking_service.client: 🏃 View run Random Forest at: http://127.0.0.1:5000/#/experiments/68

In [13]:
# Register the model
model_name = 'XGB-Smote'
run_id = input("Enter run ID:")
model_uri = f"runs:/{run_id}/model"

result = mlflow.register_model(
    model_uri, model_name
)

Successfully registered model 'XGB-Smote'.
2024/09/21 12:31:56 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB-Smote, version 1
Created version '1' of model 'XGB-Smote'.


In [14]:
# Load model
model_name = 'XGB-Smote'
model_version = 1
model_uri = f'models:/{model_name}/{model_version}'

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test) 
y_pred[:4]

array([0, 0, 0, 0])